### GENERATE A RANDOM BPMN

In [ ]:
import requests
import re
import random
import sys
sys.path.append('src')
from paco.parser.parse_tree import ParseTree
from paco.execution_tree.execution_tree import ExecutionTree
from paco.explainer.bdd.bdds import bdds_from_json

headers = {
	"Content-Type": "application/json",
}
url = "http://127.0.0.1:8000/"

### Define and Check BPMN

In [ ]:
from utils.env import EXPRESSION, IMPACTS, DURATIONS, IMPACTS_NAMES, PROBABILITIES, DELAYS, LOOP_PROBABILITY, LOOP_ROUND, H
import graphviz
from IPython.display import display, SVG

expression = '(((((((T1,T2)/[C1]((T3,T4)||T5)),((T6,T7)^[N1]T8)),((T9/[C2]T10),(T11,((T12,T13),T14)))),(((T15/[C3]T16)^[N3]T17)^[N2](T18,T19)))/[C4]((((T20,T21),T22)||T23),((T24,T25)/[C5]T26)))||((T27||((T28^[N4]T29)^[N5](T30/[C6](((T31,T32),((T33^[N7]T34)/[C7]T35)),(T36,T37)))))||T38))'

impacts_names = ["cost", "CO2", "boh", "boh2"]

impacts_range = [1, 50]
duration_range = [1, 100]
delay_range = [0, 10]

# Consider a task each T
tasks = sorted(set(re.findall(r'T\d+', expression)))
natures = sorted(set(re.findall(r'N\d+', expression)))
choices = sorted(set(re.findall(r'C\d+', expression)))
bpmn = {
	EXPRESSION: expression,
	IMPACTS: {task: [random.randint(impacts_range[0], impacts_range[1]) for _ in impacts_names] for task in tasks},
	DURATIONS: {task: [1, random.randint(duration_range[0], duration_range[1])] for task in tasks},
	IMPACTS_NAMES: impacts_names,
	DELAYS: {choice: random.randint(delay_range[0], delay_range[1]) for choice in choices},
	PROBABILITIES: {nature: round(random.uniform(0.1, 0.9), 2) for nature in natures},
	LOOP_PROBABILITY : {}, LOOP_ROUND: {}, H: 0,
}

try:
	resp = requests.get(f'{url}create_bpmn', json={'bpmn': bpmn},  headers=headers)
	resp.raise_for_status()
	display(SVG(graphviz.Source(resp.json()['bpmn_dot']).pipe(format="svg")))

except requests.exceptions.HTTPError as e:
	print(f"HTTP Error ({resp.status_code}):", resp.json())

### Create the Execution tree

In [ ]:
try:
	resp = requests.get(f'{url}create_execution_tree', json={"bpmn": bpmn}, headers=headers)
	resp.raise_for_status()

	response = resp.json()
	parse_tree, pending_choices, pending_natures = ParseTree.from_json(
		response['parse_tree'],
		impact_size=len(bpmn[IMPACTS_NAMES]),
		non_cumulative_impact_size=0)
	execution_tree = ExecutionTree.from_json(parse_tree, response["execution_tree"], bpmn[IMPACTS_NAMES])
	dot = execution_tree.to_dot(state=True, executed_time=True, diff=True)
	display(SVG(graphviz.Source(dot).pipe(format="svg")))

except requests.exceptions.HTTPError as e:
	print(f"HTTP Error ({resp.status_code}):", resp.json())

### Search using Refinements Algorithm
#### Create Execution Tree
#### Found Strategy
#### Explain Strategy

In [ ]:
import numpy as np


def search(bpmn, bound, parse_tree, execution_tree, search_only=False):
	resp = requests.get(f'{url}create_strategy', json={"bpmn": bpmn, "bound": bound, "parse_tree": parse_tree.to_dict(), "execution_tree": execution_tree.to_dict(), "search_only": search_only}, headers=headers)

	resp.raise_for_status()

	response = resp.json()

	regular_expression = r'\[([^\[\]]+)\]'
	matches = re.findall(regular_expression,
						 response["possible_min_solution"])

	new_possible_min_solution = [np.fromstring(match, sep=' ', dtype=np.float64) for match in matches]

	matches = re.findall(regular_expression,
						 response["guaranteed_bounds"])
	new_min_solutions = [np.fromstring(match, sep=' ', dtype=np.float64) for match in matches]

	if "expected_impacts" in response.keys():
		expected_impacts = np.fromstring(response["expected_impacts"].strip('[]'), sep=' ', dtype=np.float64)

		return response, expected_impacts,new_possible_min_solution,new_min_solutions
	else:
		return response, None, new_possible_min_solution,new_min_solutions

In [ ]:
from paco.evaluations.pareto import get_dominated_vectors, get_max_dominating_vectors, get_min_dominated_impacts


def found_strategy(bpmn, parse_tree, max_bound:np.ndarray= None, decimal_number: int = 0):
	min_solutions = []
	possible_min_solution = []
	if max_bound is None:
		possible_min_solution.append(np.zeros(len(bpmn[IMPACTS_NAMES]), dtype=np.float64))
	else:
		possible_min_solution.append(max_bound)

	i = 0
	found_optimal = False

	while True:
		s = ""
		for j in range(len(min_solutions)):
			s += f"Impacts {j}:\t{min_solutions[j]}\n"
		#print(f"Guaranteed Bound:\n", s)

		s = ""
		for j in range(len(possible_min_solution)):
			s += f"Impacts {j}:\t{possible_min_solution[j]}\n"
		#print(f"Possible Bound:\n", s)


		if len(possible_min_solution) > 0:
			bounds = possible_min_solution.pop(random.randint(0, len(possible_min_solution)-1)).tolist()
		elif len(min_solutions) > 0:
			found_optimal = True
			print("Min solutions found")
			bounds = min_solutions.pop(random.randint(0, len(min_solutions)-1)).tolist()
		else:
			raise Exception("No solutions found")


		print(f"Attempt {i}:\t{bpmn[IMPACTS_NAMES]}\nSelected:\t{bounds}\n")


		result, expected_impacts, new_possible_min_solution,new_min_solutions = search(bpmn, bounds, parse_tree, execution_tree, search_only=not found_optimal)
		
		#print("Search: ", result["result"])
		print("New min solution: ", new_possible_min_solution)
		print("Min solutions: ", new_min_solutions)

		if found_optimal:
			print("Optimal solution found after", i, "attempts")
			break


		#a: Finds the vectors in new_possible_min_solution dominated by vectors in new_min_solutions.
		#b: Finds the vectors in new_min_solutions dominated by vectors in new_possible_min_solution.
		a, b = get_dominated_vectors(new_possible_min_solution, new_min_solutions)
		if len(b) > 0:
			print("Limit found")
			for ei in b:
				print("Limit:", ei)
		else:
			possible_min_solution.extend([np.round(ei, decimal_number) for ei in new_possible_min_solution])
			min_solutions.extend([np.round(ei, decimal_number) for ei in new_min_solutions])
			if expected_impacts is not None:
				min_solutions.append(expected_impacts)

		possible_min_solution = get_max_dominating_vectors(possible_min_solution)
		min_solutions = get_min_dominated_impacts(min_solutions)
		i += 1

	#TODO
	if "strategy_tree" in result:
		explained_choices = bdds_from_json(parse_tree, result["bdds"])
		print("1 is dashed line of BPMN or Parse Tree")
		for choice, bdd in explained_choices.items():
			print(f"{choice.name} : {bdd.typeStrategy}")
			svg_data = graphviz.Source(bdd.bdd_to_dot()).pipe(format="svg")
			display(SVG(svg_data))

		strategy_tree = ExecutionTree.from_json(parse_tree, result["strategy_tree"], bpmn["impacts_names"], explained_choices)

		dot = strategy_tree.to_dot(state=True, executed_time=False, diff=True)
		display(SVG(graphviz.Source(dot).pipe(format="svg")))

	return result


try:
	result = found_strategy(bpmn, parse_tree)
except requests.exceptions.HTTPError as e:
	print(f"HTTP Error ({resp.status_code}):", resp.json())